# Retrieval-Augmented Generation (RAG)

In [76]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [77]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook
client = chromadb.PersistentClient("../chroma")
nutrition_db = client.get_collection("nutrition_db")
nutrition_qna = client.get_collection("nutrition_qna")
#nutrition_qna

In [78]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [27]:
results_nut = nutrition_qna.query(query_texts=["pregnancy"], n_results=2)
for i, doc in enumerate(results_nut["documents"][0]):
    print(sorted(results_nut["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', True)]
Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?
        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', True)]
Question: What health issues can make a pregnancy more challenging?
        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.

        This Q&A pair provides information about nutrition and health topics.




In [87]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [86]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to answer any questions related to nutrition and pragnancy.

    Args:
        query: The question being asked.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the answer .
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):        
        formatted_results.append(doc)

    return "The answer is :\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [11]:
calorie_lookup_tool('bananas')

'Nutrition Information:\nBanana (Fruits): 89.0 calories per 100g\nBanana Juice ((Fruit)Juices): 50.0 calories per 100g\nBanana Nut Bread (Pastries,Breads&Rolls): 326.0 calories per 100g'

In [14]:
nutrition_qna_tool('pragnancy')

"The answer is :\nQuestion: What population does POHSHAN HABIAHNT primarily aim its efforts towards?\n        Answer: The main focus of POHSAN HABIYAHNT is on improving the nutritional status of adolescent girls, pregnant women, and young children.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: What group does POSHAN Abhiyaan prioritize?\n        Answer: POSHAN Abhiyaan's priority is the nutritional health of adolescent girls, pregnant women, lactating mothers, and children from 0 to 6 years old.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: In terms of the behavioral change model, where does the woman in Case 1 think she stands?\n        Answer: According to Case 1, the woman thinks she's in the contemplation and preparation stages, as she's pondering over changing her feeding practices.\n\n        This Q&A pair provides information about nutrition and health topics."

In [ ]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful assistant giving out calorie information and nutrition advice.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.    
    List the tools used
    """,
    tools=[calorie_lookup_tool]
)

In [88]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are there in one banana and an apple?",
    )
    print(result.final_output)

Approximate total: about 200 calories.

- Banana (medium, ~118 g): ~105 kcal
- Apple (medium, ~182 g): ~95 kcal
- Total: ~200 kcal
